In [1]:
from CodeGenerators import codegen
import json, subprocess, re, pyodbc, os
from sqlalchemy import func, create_engine
import numpy as np # linear algebra 
from config import mongoConn, connstr
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
import pandas as pd 

In [79]:
engine = create_engine(connstr)
conn = engine.connect() 
query = """ \
	SELECT DISTINCT PK_FORM, Report_Year, FK_ReportingCycle, Description AS TAB, FORM_Description AS FDESC   
    FROM fsma_FormMaster 
    INNER JOIN fsma_ReportingCycles ON fsma_FormMaster.FK_ReportingCycle = fsma_ReportingCycles.PK_ReportingCycle 
    WHERE Report_Year >= 2019 
    ORDER BY FK_ReportingCycle DESC 
""" 
df = pd.read_sql(query,con=conn)     
conn.close()

In [80]:
with open(r"D:\dev\CyberScope\CyberScopeBranch\CSwebdev\test\bom\config.yaml") as f: 
    tpl = f.read()   
for i, r in df.iterrows():  
    c = tpl
    PK_FORM = re.sub('-','_',r.PK_FORM)
    c = c.replace('#PK_FORM', PK_FORM)
    c = c.replace('#TAB', re.sub('/','.*',r.TAB))
    c = c.replace('FDESC', re.sub('-','_',r.FDESC)) 
    with open(f"D:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\test\\bom\\init\\{PK_FORM}.yaml", "w+") as f: 
        f.write(c)

In [77]:
# df.head(100)